# Challenge 1: Properti

## Goals:
- Clean the supplied dataset, designing strategies to deal with lost data in certain variables. 
- Perform a descriptive analysis of the main variables.
- Create new columns based on the given characteristics that may have predictive value.

### Import Libraries
**Numpy** and **Pandas** libraries are used to load and manage the information supplied in the csv file.

**IPython** is used to perform the presentation of the information obtained durign the analysis. 

In [23]:
import numpy as np
import pandas as pd
from IPython.display import display

### Load CSV file
Load the dataset specifiying the comma as the delimiter value .

In [24]:
dfprop=pd.read_csv(filepath_or_buffer='properatti.csv',delimiter=',')

### Visualize Strucure Information
#### Dataset Structure (Columns - data type)


In [30]:
#ar_columns=np.array(dfprop.columns)
#ar_columns.info()
dfprop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121220 entries, 0 to 121219
Data columns (total 26 columns):
Unnamed: 0                    121220 non-null int64
operation                     121220 non-null object
property_type                 121220 non-null object
place_name                    121197 non-null object
place_with_parent_names       121220 non-null object
country_name                  121220 non-null object
state_name                    121220 non-null object
geonames_id                   102503 non-null float64
lat-lon                       69670 non-null object
lat                           69670 non-null float64
lon                           69670 non-null float64
price                         100810 non-null float64
currency                      100809 non-null object
price_aprox_local_currency    100810 non-null float64
price_aprox_usd               100810 non-null float64
surface_total_in_m2           81892 non-null float64
surface_covered_in_m2         101313 no

### Columns definition
- Unnamed: 0:

- operation:

- property_type:

- place_name:

- place_with_parent_names:

- country_name:

- state_name:

- geonames_id:

- lat-lon:

- lat:

- lon:

- price:

- currency:

- price_aprox_local_currency:

- price_aprox_usd:

- surface_total_in_m2:

- surface_covered_in_m2:

- price_usd_per_m2:

- price_per_m2:

- floor:

- rooms:

- expenses:

- properati_url:

- description:

- title:

- image_thumbnail:


## Nan values per variable

In [51]:
dfprop[dfprop.isnull().any(axis=1)].isna().sum()

Unnamed: 0                         0
operation                          0
property_type                      0
place_name                        23
place_with_parent_names            0
country_name                       0
state_name                         0
geonames_id                    18717
lat-lon                        51550
lat                            51550
lon                            51550
price                          20410
currency                       20411
price_aprox_local_currency     20410
price_aprox_usd                20410
surface_total_in_m2            39328
surface_covered_in_m2          19907
price_usd_per_m2               52603
price_per_m2                   33562
floor                         113321
rooms                          73830
expenses                      106958
properati_url                      0
description                        2
title                              0
image_thumbnail                 3112
dtype: int64